In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize confi||g with toml file
c = sfp.SolutionFilesConfig('config_files/china/CHN.toml')

# c = sfp.SolutionFilesConfig('config_files/thailand/THA.toml')
ix = pd.IndexSlice  

from solution_file_processing.plots import create_plot_1a, create_plot_2b_ref_plots, create_plot_2, create_plot_10_ts_by_model
from solution_file_processing.plots import create_plot_6_ldc, create_plot_7_co2_savings, create_plot_8_services, create_plot_9_av_cap

[18:09:18 00:00] I:119:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240903_180918-CHN.log.
[18:09:18 00:00] I:156:__init__ - Initialized SolutionFilesConfig for config_files/china/CHN.toml.


In [ ]:
purch_df = c.o.purch_df
dsm_profiles_ts = purch_df[
                purch_df.name.str.contains('_Shift') & (purch_df.property == 'Load')].groupby(
                ['model', 'timestamp']).sum()


In [ ]:
df = dd.from_pandas(pd.DataFrame(None), npartitions=1)
df = df.compute()
df.shape[0]


In [ ]:
x = purch_df.loc[1,:].compute()

In [ ]:
dsm_profiles_orig_ts = x[
        x.name.str.contains('_Shift') & (x.property == 'x')].groupby(
        ['model', 'timestamp']).agg({'value': 'sum'})

In [ ]:
purch_df.shape[0].compute()

In [ ]:
"""
DSM profiles
"""
# Model filler for comparison of models with different inputs (e.g. DSM or EVs not included)
# Series with indices matching the columns of the DF for filling in missing columns

model_filler = pd.Series(data=[1] * len(c.v.model_names), index=c.v.model_names).rename_axis('model')

len(c.o.purch_df)

In [ ]:
if purch_df.shape[0] > 0:
    dsm_profiles_orig_ts = purch_df[
        purch_df.name.str.contains('_Shift') & (purch_df.property == 'x')].groupby(
        ['model', 'timestamp']).sum()
    if not dsm_profiles_orig_ts.shape[0] == 0:
        dsm_profiles_orig_ts = (dsm_profiles_orig_ts.value.unstack('model') * model_filler).fillna(0).stack('model').reorder_levels(
            ['model', 'timestamp'])
    else:
        dsm_profiles_orig_ts = pd.DataFrame(data=[0] * len(self.c.v.customer_load_ts.index),
                                index=self.c.v.customer_load_ts.index, columns=['value'])
else:
    dsm_profiles_orig_ts = pd.DataFrame(data=[0] * len(self.c.v.customer_load_ts.index),    
                                index=self.c.v.customer_load_ts.index, columns=['value'])

In [ ]:
from zipfile import ZipFile
from pathlib import Path
import os
import fnmatch

def extract_plexos_LT_results(model_path):
    
    if not isinstance(model_path, Path):
        model_path = Path(model_path)

    exp_out_path = model_path / '03_Modelling/01_InputData/08_ExpUnits/'
    lt_solns_path = exp_out_path / 'LT_solution_files/'
    lt_soln_zips = [ lt_solns_path / f for f in  os.listdir(lt_solns_path)]

    for z in lt_soln_zips:
        with ZipFile(z, 'r') as zipObj:
            file_list = zipObj.namelist()
            pattern = '* Units.csv'

            filtered_list = []
            for file in file_list:
                if fnmatch.fnmatch(file, pattern):
                    filtered_list.append(file)
            
            for f in filtered_list:
                try:
                    zipObj.extract(f, exp_out_path)
                    print(f'Extracted {f} to {exp_out_path}')
                except PermissionError:
                    print(f'Could not extract {f} to {exp_out_path}. File already exists.')
                    continue
            
            # Create a list of all the extracted files for Generator Units
            # This can then be used to increase the capacity of the units by 5% and integerise the capacity
            exp_units_files = [ exp_out_path / f for f in  os.listdir(exp_out_path) if 'Units.csv' in f]

            for f in exp_units_files:
                df = pd.read_csv(f)
                df['Value'] = df['Value'] * 1.05
                df.loc[df.Name.str.contains('Gas'),'Value'] = df['Value'].apply(np.round)
                try:
                    df.to_csv(f, index=False)
                except PermissionError:
                    print(f'Could not write to {f}. File already exists.')
                    continue


In [ ]:
model_path = Path('Y:/Modelling/Ukraine/2023_UKR_ST_Security/')
extract_plexos_LT_results(model_path=model_path)